In [1]:
import polars as pl
from cleaner import sanitize
from geopy import distance
import math
import pydeck

In [2]:
# df = pl.read_parquet("data/processed.parquet")

In [3]:
# df.shape

In [4]:
# df.head()

In [5]:
# df = df.with_columns(pl.col("crash_date").str.to_date("%m/%d/%Y").alias("date"))
# df = df.with_columns(pl.col("date").dt.year().alias("year"))
# df = df.with_columns(pl.col("date").dt.month().alias("month"))
# df = df.with_columns(pl.col("crash_time").str.to_time("%H:%M").alias("time"))
# df = df.with_columns(pl.col("time").dt.hour().alias("hour"))

# # df[["date", "year", "month", "time", "hour"]].head()

In [6]:
# df = df.drop(["crash_date", "crash_time", "location"])
# df = df.drop('collision_id')
# df.write_parquet('data/processed.parquet', use_pyarrow=True)

In [7]:
# df[["date", "borough", "year", "month", "time", "hour"]].write_parquet("data/time_only.parquet", use_pyarrow=True)

In [8]:
clean = pl.read_parquet("data/clean_data.parquet")

In [9]:
# clean = df.drop_nulls(subset=["latitude"])
# clean = clean.drop_nulls(subset=["longitude"])
# clean = clean.drop_nulls(subset=["borough"])

In [10]:
# clean.shape

In [11]:
clean = clean.with_columns((pl.concat_list(pl.col("latitude"), pl.col("longitude")).alias("coordinate")))

In [12]:
# clean = clean.with_columns(pl.lit((40.7128, -74.0060)).alias('ny'))
# clean.head()

In [13]:
# clean = clean.with_columns((pl.concat_list(pl.col("coordinate"), pl.col("ny")).alias("geodesic")))
# clean.head()

In [14]:
# clean['geodesic'].dtype

In [15]:
# clean = clean.with_columns(pl.col("geodesic").map_elements(lambda x: distance.geodesic((x[0], x[1]), (x[2], x[3])).km, return_dtype=float).alias("distance"))
# clean.head()

In [16]:
# clean = clean.drop(["geodesic", "ny", "coordinate"])
clean.head()

borough,zip_code,latitude,longitude,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,date,year,month,time,hour,distance,color,coordinate
str,i64,f64,f64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,date,i32,i8,time,i8,f64,str,list[f64]
"""BROOKLYN""",11208,40.667202,-73.8665,null,null,"""1211 LORING AVENUE""",0,0,0,0,0,0,0,0,"""Unspecified""",null,null,null,null,"""Sedan""",null,null,null,null,2021-09-11,2021,9,09:35:00,9,12.832897,"""#0044ff""","[40.667202, -73.8665]"
"""BROOKLYN""",11233,40.683304,-73.917274,"""SARATOGA AVENUE""","""DECATUR STREET""",null,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,2021-12-14,2021,12,08:13:00,8,8.183083,"""#0044ff""","[40.683304, -73.917274]"
"""BRONX""",10475,40.86816,-73.83148,null,null,"""344 BAYCHESTER AVENUE""",2,0,0,0,0,0,2,0,"""Unspecified""","""Unspecified""",null,null,null,"""Sedan""","""Sedan""",null,null,null,2021-12-14,2021,12,08:17:00,8,22.685206,"""#2ca02c""","[40.86816, -73.83148]"
"""BROOKLYN""",11207,40.67172,-73.8971,null,null,"""2047 PITKIN AVENUE""",0,0,0,0,0,0,0,0,"""Driver Inexperience""","""Unspecified""",null,null,null,"""Sedan""",null,null,null,null,2021-12-14,2021,12,21:10:00,21,10.273211,"""#0044ff""","[40.67172, -73.8971]"
"""MANHATTAN""",10017,40.75144,-73.97397,"""3 AVENUE""","""EAST 43 STREET""",null,0,0,0,0,0,0,0,0,"""Passing Too Closely""","""Unspecified""",null,null,null,"""Sedan""","""Station Wagon/Sport Utility Ve…",null,null,null,2021-12-14,2021,12,14:58:00,14,5.07277,"""#ffaa00""","[40.75144, -73.97397]"


In [17]:
clean.shape

(1411078, 33)

In [18]:
clean['distance'].describe()

statistic,value
str,f64
"""count""",1.411078e6
"""null_count""",0.0
"""mean""",11.839088
"""std""",6.127136
"""min""",0.078833
"""25%""",7.035497
"""50%""",11.337957
"""75%""",16.337666
"""max""",31.27159


In [19]:
# clean.filter(pl.col('distance') > 530).write_parquet("data/invalid_coordinate.parquet")
# clean = clean.filter(pl.col("distance") < threshold)

In [20]:
# threshold = 1.5*clean['distance'].mean()
# clean = clean.filter(pl.col("distance") < threshold)
clean.head()

borough,zip_code,latitude,longitude,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,date,year,month,time,hour,distance,color,coordinate
str,i64,f64,f64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,date,i32,i8,time,i8,f64,str,list[f64]
"""BROOKLYN""",11208,40.667202,-73.8665,null,null,"""1211 LORING AVENUE""",0,0,0,0,0,0,0,0,"""Unspecified""",null,null,null,null,"""Sedan""",null,null,null,null,2021-09-11,2021,9,09:35:00,9,12.832897,"""#0044ff""","[40.667202, -73.8665]"
"""BROOKLYN""",11233,40.683304,-73.917274,"""SARATOGA AVENUE""","""DECATUR STREET""",null,0,0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,2021-12-14,2021,12,08:13:00,8,8.183083,"""#0044ff""","[40.683304, -73.917274]"
"""BRONX""",10475,40.86816,-73.83148,null,null,"""344 BAYCHESTER AVENUE""",2,0,0,0,0,0,2,0,"""Unspecified""","""Unspecified""",null,null,null,"""Sedan""","""Sedan""",null,null,null,2021-12-14,2021,12,08:17:00,8,22.685206,"""#2ca02c""","[40.86816, -73.83148]"
"""BROOKLYN""",11207,40.67172,-73.8971,null,null,"""2047 PITKIN AVENUE""",0,0,0,0,0,0,0,0,"""Driver Inexperience""","""Unspecified""",null,null,null,"""Sedan""",null,null,null,null,2021-12-14,2021,12,21:10:00,21,10.273211,"""#0044ff""","[40.67172, -73.8971]"
"""MANHATTAN""",10017,40.75144,-73.97397,"""3 AVENUE""","""EAST 43 STREET""",null,0,0,0,0,0,0,0,0,"""Passing Too Closely""","""Unspecified""",null,null,null,"""Sedan""","""Station Wagon/Sport Utility Ve…",null,null,null,2021-12-14,2021,12,14:58:00,14,5.07277,"""#ffaa00""","[40.75144, -73.97397]"


In [21]:
clean.shape

(1411078, 33)

In [22]:
colors = [
    '#0044ff',  # muted blue
    '#ffaa00',  # muted purple
    '#00ff00',  # chestnut brown
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    
]
boroughs = clean['borough'].unique()
colors = colors[:boroughs.shape[0]]

In [23]:
# color_lookup = pydeck.data_utils.assign_random_colors(clean['borough'])
# colors = list(color_lookup.values())

In [24]:
# clean = clean.with_columns(pl.col("borough").map_elements(lambda x: color_lookup[x], return_dtype=list[int]).alias("color"))
# clean = clean.with_columns(pl.col("borough").str.replace_many(boroughs, colors).alias("color"))

In [25]:
# clean.write_parquet("data/clean_data.parquet", use_pyarrow=True)

In [26]:
clean.select(["date", "borough", "latitude", "longitude", "month", "year", "time", "hour", "coordinate", "color"]).write_parquet("data/clean_map.parquet", use_pyarrow=True)